In [ ]:
import pandas as pd

# Import matplotlib and setting aesthetics for plotting 
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
plt.style.use('fivethirtyeight')

# Read datasets/coinmarketcap_06122017.csv into pandas
dec6 = pd.read_csv('datasets/coinmarketcap_06122017.csv')

# Select the 'id' and the 'market_cap_usd' columns
market_cap_raw = dec6[['id','market_cap_usd']]

# Counting the number of values
market_cap_raw.count()
print(market_cap_raw.head())
print(market_cap_raw.tail())

In [ ]:
# Filter out rows without a market capitalization
cap = market_cap_raw.query('market_cap_usd > 0')
print(cap.tail())

# Count the number of values again
cap.count()

In [ ]:
# Declare these now for later use in the plots
TOP_CAP_TITLE = 'Top 10 market capitalization'
TOP_CAP_YLABEL = '% of total cap'

# Select the first 10 rows and set the index
cap10 = cap.head(10).set_index(cap.id[:10])

# Calculate market_cap_perc
cap10 = cap10.assign(market_cap_perc = lambda x: (x.market_cap_usd/cap.market_cap_usd.sum())*100)

# Plot the barplot with the title defined above 
ax = cap10.plot.bar(x='id', y='market_cap_perc', title=TOP_CAP_TITLE)

# Annotate the y axis with the label defined above
ax.set_ylabel(TOP_CAP_YLABEL)

In [ ]:
# Colors for the bar plot
COLORS = ['orange', 'green', 'orange', 'cyan', 'cyan', 'blue', 'silver', 'orange', 'red', 'green']

# Plot market_cap_usd as before but adding the colors and scaling the y-axis  
ax = cap10.plot.bar(x='id', y='market_cap_usd', title=TOP_CAP_TITLE, color = COLORS, logy=True)

# Annotating the y axis with 'USD'
ax.set_ylabel('USD')

# Remove the xlabel as it is not very informative
ax.set_xlabel('')

In [ ]:
# Select the id, percent_change_24h and percent_change_7d columns
volatility = dec6[['id', 'percent_change_24h', 'percent_change_7d']]

# Set the index to 'id' and drop all NaN rows
volatility = volatility.set_index('id').dropna()

# Sort the DataFrame by percent_change_24h in ascending order
volatility = volatility.sort_values('percent_change_24h', axis=0, ascending=True)

# Check the first few rows
volatility.head()

In [ ]:
#Define a function with 2 parameters, the series to plot and the title
def top10_subplot(volatility_series, title):
    # Make the subplot and the figure for two side by side plots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))
    
    # Plote with pandas the barchart for the top 10 losers
    ax = volatility_series[:10].plot.bar(color='darkred', ax=axes[0])
    
    # Set the figure's main title to the text passed as parameter
    fig.suptitle(title)
    
    
    # Set the ylabel to '% change'
    ax.set_ylabel('% change')
    
    
    # Same as above, but for the top 10 winners
    ax = volatility_series[-10:].plot.bar(color='darkblue', ax=axes[1])
    
    # Return this for good practice
    return fig, ax

DTITLE = "24 hours top losers and winners"

# Call the function above with the 24 hours period series and title DTITLE  
fig, ax = top10_subplot(volatility.percent_change_24h, DTITLE)

In [ ]:
# Sort in ascending order
volatility7d = volatility.sort_values('percent_change_7d',ascending=True)
print(volatility7d.head())

WTITLE = "Weekly top losers and winners"

# Call the top10_subplot function
fig, ax = top10_subplot(volatility7d.percent_change_7d, WTITLE)

In [ ]:
# Select everything bigger than 10 billion 
largecaps = cap.query('market_cap_usd > 10e+09')

# Print out largecaps
print(largecaps)

In [ ]:
# Make a function for counting different marketcaps from the
# "cap" DataFrame. Returns an int.
def capcount(query_string):
    return cap.query(query_string).count().id

# Labels for the plot
LABELS = ["biggish", "micro", "nano"]

# Use capcount count the biggish cryptos
biggish = capcount('market_cap_usd > 3e+08')

# Same as above for micro 

micro = capcount('market_cap_usd > 50e+06 & market_cap_usd < 3e+08')

# ... and for nano
nano =  capcount('market_cap_usd < 50e+06')

# Make a list with the 3 counts
values = [biggish, micro, nano]

# Plot them with matplotlib 
plt.bar(range(len(values)), values, tick_label = LABELS)